# Here, I am creating a new column in the goalie_hist_model_ready DF that creates a Goalie Score metric for analysis. 

In [37]:
import pandas as pd
import numpy as np
import requests
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler


In [38]:
#Import goalie-specific, model ready csv

goalie_hist_model_ready = pd.read_csv("C:/Users/Daniel Quinn/Desktop/Bootcamp/Project_2/data/processed/goalie_hist_model_ready.csv")
#Ensure we can read the whole dataframe, without "..."
pd.set_option("display.max_rows", None)
goalie_hist_model_ready = goalie_hist_model_ready.drop(columns = ['Unnamed: 0'])
goalie_hist_model_ready.head()


,id,first_name,second_name,team,element_type,code,element_code,season,start_cost,end_cost,...,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded
0,5,Karl,Hein,1,1,463748,463748,2021,40,40,...,0,0.0,0.0,0,0.0,0,0.00,0.00,0.00,0.00
1,5,Karl,Hein,1,1,463748,463748,2022,40,39,...,0,0.0,0.0,0,0.0,0,0.00,0.00,0.00,0.00
2,5,Karl,Hein,1,1,463748,463748,2023,40,40,...,0,0.0,0.0,0,0.0,0,0.00,0.00,0.00,0.00
3,15,David,Raya Martin,1,1,154561,154561,2021,45,44,...,496,593.4,10.0,0,60.1,0,0.00,0.00,0.00,0.00
4,15,David,Raya Martin,1,1,154561,154561,2022,45,48,...,822,1146.0,20.1,4,117.3,38,0.11,0.12,0.23,50.12


In [39]:
#Create Saves Percentage stat:

goalie_hist_model_ready["Saves_Percentage"] = (
    (goalie_hist_model_ready['saves'] + goalie_hist_model_ready['penalties_saved']) /
    (goalie_hist_model_ready['saves'] + goalie_hist_model_ready['penalties_saved'] + goalie_hist_model_ready['goals_conceded'])
) * 100
#pd.set_option("display.max_columns", None)
goalie_hist_model_ready.shape

(269, 34)

In [40]:
# Prep dataframe for modeling


goalie_hist_model_ready_cleaned = goalie_hist_model_ready.drop(columns = ['season','first_name', 'second_name', 'team',  ])
# goalie_hist_model_ready.columns
goalie_hist_model_1_ready = goalie_hist_model_ready_cleaned[['id', 'element_type', 'code',
       'element_code', 'start_cost', 'end_cost', 'total_points', 'minutes',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'saves', 'bonus', 'bps', 'ict_index', 'starts',
       'expected_goals', 'expected_assists', 'expected_goal_involvements',
       'expected_goals_conceded', 'Saves_Percentage']]

dropna_goalie_hist_model_1_ready = goalie_hist_model_1_ready.dropna()
dropna_goalie_hist_model_1_ready.head()

,id,element_type,code,element_code,start_cost,end_cost,total_points,minutes,assists,clean_sheets,...,saves,bonus,bps,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,Saves_Percentage
3,15,1,154561,154561,45,44,95,2160,0,8,...,78,5,496,60.1,0,0.00,0.00,0.00,0.00,74.285714
4,15,1,154561,154561,45,48,166,3420,0,12,...,154,20,822,117.3,38,0.11,0.12,0.23,50.12,77.000000
5,15,1,154561,154561,50,53,135,2880,0,16,...,46,6,569,41.1,32,0.00,0.04,0.04,22.51,66.197183
6,73,1,69752,69752,45,45,93,2385,0,6,...,100,9,510,72.8,27,0.00,0.02,0.02,44.70,71.942446
7,73,1,69752,69752,45,46,110,2880,0,7,...,113,14,592,87.5,32,0.00,0.01,0.01,49.66,67.455621


In [41]:
#Create a better dataframe name for going forward

goalie_hist_model_2_ready = dropna_goalie_hist_model_1_ready

In [42]:
#create X & y variables
X = goalie_hist_model_2_ready.drop(columns = ['end_cost'])
y = goalie_hist_model_2_ready['end_cost']

#test-training split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#create model
model = LinearRegression()

model.fit(X_train, y_train)
print("Train model score: ", model.score(X_train, y_train))
print("Test model score: ", model.score(X_test, y_test))

Train model score:  0.9283766153574129
Test model score:  0.8556570365023447


In [43]:
#Make predictions

prediction1 = model.predict(X_test)

#Evaluate models with mse and r2

mse = mean_squared_error(y_test, prediction1) # how close are the predicted values to actual values via the squared differences between expected and real
r2 = r2_score(y_test, prediction1) # r2 - how well do the indep variables explain the variation in the dep var? ) 0 is a perfect model, the larger the nuber, the worse the model is performing

print(f"All Features (Saves_Percentage = y):")
print(f"mean squared error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

All Features (Saves_Percentage = y):
mean squared error (MSE): 4.193586022675736
R-squared (R2): 0.8556570365023447


In [44]:
#Run the model to get the predicted Goalie Score

predicted_Saves_Percentage = goalie_hist_model_1_ready.drop(columns = ['season_start_date', 'Goalie_Score', 'first_name', 'second_name'])
Goalie_Saves_Percentage = model.predict(predicted_Goalie_Score)
print(Goalie_Saves_Percentage)

KeyError: "['season_start_date', 'Goalie_Score', 'first_name', 'second_name'] not found in axis"

In [42]:
#create X & y variables
X = goalie_hist_model_1_ready.drop(columns=['season_start_date', 'first_name', 'second_name', 'Goalie_Score'])
y = goalie_hist_model_1_ready['Goalie_Score']

#test-training split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#create & train model
random_forest = RandomForestRegressor(n_estimators=500, random_state=42).fit(X_train, y_train)

# Evaluate the model
print(f"Training Score: {random_forest.score(X_train, y_train)}")
print(f"Testing Score: {random_forest.score(X_test, y_test)}")

Training Score: 0.9996168755316256
Testing Score: 0.9974507584902772


In [43]:
# Feature Importance
feature_importances = random_forest.feature_importances_

feature_importances_df = pd.DataFrame(feature_importances, X.columns)


print(feature_importances_df.sort_values(by=0, ascending=False))

                                   0
bps                         0.981318
ict_index                   0.009356
total_points                0.006314
saves                       0.000846
minutes                     0.000679
id                          0.000238
bonus                       0.000220
code                        0.000218
element_code                0.000153
team                        0.000140
goals_conceded              0.000136
clean_sheets                0.000081
end_cost                    0.000079
starts                      0.000048
start_cost                  0.000044
penalties_saved             0.000040
expected_goals_conceded     0.000039
expected_assists            0.000016
expected_goal_involvements  0.000014
expected_goals              0.000009
own_goals                   0.000007
assists                     0.000005
element_type                0.000000
